In [8]:
import numpy as np
import time
import cv2
from pygame import mixer


def machine_state(sumation,sound):
    # Check if blue color object present in the ROI 	
    yes = (sumation) > Pan_thickness[0]*Pan_thickness[1]*0.8

    # If present play the respective instrument.
    if yes and sound==1:
        clap.play()

    elif yes and sound==2:
        drum.play()
        time.sleep(0.001)

def analysis_ROI(frame,sound):
    # converting the image into HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # generating mask for 
    mask = cv2.inRange(hsv, blueLower, blueUpper)
    
    # Calculating the nuber of white pixels depecting the blue color pixels in the ROI
    sumation = np.sum(mask)
    
    # Function that decides to play the instrument or not.
    machine_state(sumation,sound)

    return mask


Verbsoe = False

# importing the audio files
def audio_file():
    mixer.init()
    clap = mixer.Sound(r'C:\Users\Admin\Desktop\graphics\AIR_Drums-master\AIR_Drums-master\batterrm.wav')
    drum = mixer.Sound(r'C:\Users\Admin\Desktop\graphics\AIR_Drums-master\AIR_Drums-master\button-2.ogg')
    
    return clap,drum


# Function to read image of Pan and Drum for augmentation.
def read_image():
    Pan = cv2.resize(cv2.imread(r'C:\Users\Admin\Desktop\graphics\AIR_Drums-master\AIR_Drums-master\Images\MetalPan.png'),
                      (200,100),interpolation=cv2.INTER_CUBIC)
    Drum = cv2.resize(cv2.imread(r'C:\Users\Admin\Desktop\graphics\AIR_Drums-master\AIR_Drums-master\Images\Drum.png'),
                  (200,100),interpolation=cv2.INTER_CUBIC)

    return Pan, Drum


    
# Function to detect ROI to view color being detected
def ROI_color_detect():
    if Verbsoe:
        # Displaying the ROI in the Image
        frame[Drum_top[1]:Drum_btm[1],Drum_top[0]:Drum_btm[0]] = cv2.bitwise_and(frame[Drum_top[1]:Drum_btm[1],Drum_top[0]:Drum_btm[0]],
                                                                                 frame[Drum_top[1]:Drum_btm[1],Drum_top[0]:Drum_btm[0]], 
                                                                                 mask=mask[Drum_top[1]:Drum_btm[1],Drum_top[0]:Drum_btm[0]])
        frame[Pan_top[1]:Pan_btm[1],Pan_top[0]:Pan_btm[0]] = cv2.bitwise_and(frame[Pan_top[1]:Pan_btm[1],Pan_top[0]:Pan_btm[0]],
                                                                             frame[Pan_top[1]:Pan_btm[1],Pan_top[0]:Pan_btm[0]],
                                                                             mask=mask[Pan_top[1]:Pan_btm[1],Pan_top[0]:Pan_btm[0]])
    
    # Augmenting the instruments in the output frame.
    else:
        # Augmenting the image of the instruments on the frame.
        frame[Drum_top[1]:Drum_btm[1],Drum_top[0]:Drum_btm[0]] = cv2.addWeighted(Drum, 1, frame[Drum_top[1]:Drum_btm[1],Drum_top[0]:Drum_btm[0]], 1, 0)
        frame[Pan_top[1]:Pan_btm[1],Pan_top[0]:Pan_btm[0]] = cv2.addWeighted(Pan, 1, frame[Pan_top[1]:Pan_btm[1],Pan_top[0]:Pan_btm[0]], 1, 0)
    


# HSV range for detecting blue color 
blueLower = (80,150,10)
blueUpper = (120,255,255)

# Frame accusition from webcam/ usbcamera 
camera = cv2.VideoCapture(0)
ret,frame = camera.read()
H,W = frame.shape[:2]

kernel = np.ones((7,7),np.uint8)

# reading the image of Pan and Drum for augmentation.
Pan, Drum = read_image()

# calling audio function
clap, drum = audio_file()

# Setting the ROI area for color detection
Pan_center = [np.shape(frame)[1]*2//8,np.shape(frame)[0]*6//8]
Drum_center = [np.shape(frame)[1]*6//8,np.shape(frame)[0]*6//8]
Pan_thickness = [200,100]
Pan_top = [Pan_center[0]-Pan_thickness[0]//2,Pan_center[1]-Pan_thickness[1]//2]
Pan_btm = [Pan_center[0]+Pan_thickness[0]//2,Pan_center[1]+Pan_thickness[1]//2]
    
Drum_thickness = [200,100]
Drum_top = [Drum_center[0]-Drum_thickness[0]//2,Drum_center[1]-Drum_thickness[1]//2]
Drum_btm = [Drum_center[0]+Drum_thickness[0]//2,Drum_center[1]+Drum_thickness[1]//2]


time.sleep(1)

while True:
    # grab the current frame
    ret, frame = camera.read()
    frame = cv2.flip(frame,1)

    if not(ret):
        break
    
    # Selecting ROI corresponding to Drum
    Drum_ROI = np.copy(frame[Drum_top[1]:Drum_btm[1],Drum_top[0]:Drum_btm[0]])
    mask = analysis_ROI(Drum_ROI,1)

    # Selecting ROI corresponding to Pan
    Pan_ROI = np.copy(frame[Pan_top[1]:Pan_btm[1],Pan_top[0]:Pan_btm[0]])
    mask = analysis_ROI(Pan_ROI,2)
    
    # A writing text on an image.
    cv2.putText(frame,'Project: Air Drums',(10,30),2,1,(20,20,20),2)
    
    # Display the ROI to view the blue colour being detected
    ROI_color_detect()
    
    cv2.imshow('Output',frame)
    key = cv2.waitKey(1) & 0xFF
    # if the 'q' key is pressed, stop the loop
    if key == ord("q"):
        break



#cleanup the camera and close any open windows
camera.release()
cv2.destroyAllWindows()